<a href="https://colab.research.google.com/github/mohitmeht/Research_codes/blob/master/Simulating_voltage_response_in_Li_air.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Plotting polarization curve using Prof. Petru Andrei's equation as discussed in (Mehta, Bevara, and Andrei, “Maximum Theoretical Power Density of Lithium–Air Batteries with Mixed Electrolyte.”) 

# Import libraries

In [0]:
import scipy.constants as sciconst
import numpy as np
import matplotlib.pylab as plt
from numba import jit,njit,vectorize
from scipy.optimize import fsolve


## Compute thermal voltage

In [0]:
@jit(parallel=True,fastmath=True)
def Vt(T=300):
  kb = (sciconst.physical_constants['Boltzmann constant'])[0]
  eV = (sciconst.physical_constants['electron volt'])[0]
  kb_div_eV = np.divide(kb,eV) 
  ThermalVoltage = np.multiply(T,kb_div_eV)
  return ThermalVoltage

## Compute $\lambda$ from the paper
\begin{equation}
\lambda = \tanh^{-1}\left(\frac{a}{b}\right)
\end{equation}
Diffusion length ($\lambda$) is based on Petru's theory to compute the electrochemical impedance spectra in Li-air/O$_2$ batteries. 
This function return a value, lambda.
$\lambda^2 = \frac{D_\textrm{eff}}{k}$ is in cm.

\begin{equation}
\lambda I_F= nFAD_\textrm{eff}c_{o_2}\tanh\left(\frac{l}{\lambda}\right)
\end{equation} or 
\begin{equation}
j_F \lambda= nF*D_\textrm{eff}c_{o_2}\tanh\left(\frac{l}{\lambda}\right)
\end{equation}
where, 

$n$ is the number of charge transfer electrons, 

$F_\textrm{const}$ is the Faradaic constant in C/mol,

$A$ is the cross sectional area of the cathode  in cm$^2$ (removed when using current density and not current),

$D\textrm{eff}$ is the effective oxygen diffusion coefficient in the organic electrolyte in cm$^2$/s,

$L_c$ = Length of the cathode in cm,

$j_{dis}$ is the value of the dc discharge current density in A/cm$^2$,

$f$ is the function used to solve the non-linear equation using a numerical receipe to find the diffusion length ($\lambda$),

$c_\textrm{int}$ is the inital concentration of oxygen dissolved in the electrolyte in mol/cm$^3$.

The diffusion length is required to compute the voltage drop due to oxygen concentration polarization.

In [0]:
# % [ lamad ] = petru_equation_current_density(n,F_const,A,Deff,c_int,Lc,j_dis);
# [ lamad ] = petru_equation_current_density(n,F_const,A,Deff,c_int,Lc,j_dis);
# x0 = -ones(size(j_dis));  % Make a starting guess at the solution

@jit(parallel=True,fastmath=True)
def f(x,n,F,Deff,c_int,Lc,j_dis):
  Fa_const = n*F*A*Deff*c_int
  tanh_func = np.tanh(np.divide(Lc,x))
  Fa = np.multiply(Fa_const,tanh_func)
  Fb = np.multiply(j_dis,x)
  # print("{}\n{}\n{}\n{}\n{}\n{}\n{}".format(n,F,A,Deff,c_int,l,I_dis))
  return np.subtract(Fa,Fb) #F = (n.*F_const.*A.*Deff.*c_int.*tanh(Lc./x))-(j_dis.*x));

@jit(parallel=True,fastmath=True)
def petru_equation(n,F,A,Deff,c_int,l,I_dis):
  x0 = -1*np.ones_like(I_dis)
  jdis = np.divide(I_dis,A)
  lam = fsolve(f,x0,args=(n,F,Deff,c_int,l,jdis))
  return lam

## Constants

In [0]:
n = 2;
F_const = (sciconst.physical_constants['Faraday constant'])[0]
A = 1.131; # in cm2
Do2 = 24e-6; # in cm2/s
porol = 0.6;
brugg = 1.5
Deff = np.multiply(Do2,np.power(porol,brugg)); # in cm2/s
c_int = 8.76e-6;# in mol/cm3
l = 12e-4; # in cm, Cathode of 12um
T = 300; # in K
a = 1.4286e5; # in 1/cm
beta = 0.5;
k0 = 0.0020e-8; # in cm/s
ja0 = 6.17e-4; # in A/cm2
I_dis_lst = np.arange(0.5,10.1,0.5)

In [0]:
I_dis = I_dis_lst[0]
lam  = petru_equation(n,F_const,A,Deff,c_int,l,I_dis)
print(lam)



2
96485.33289
1.131
1.115419203707736e-05
8.76e-06
0.0012
0.5
2
96485.33289
1.131
1.115419203707736e-05
8.76e-06
0.0012
0.5
2
96485.33289
1.131
1.115419203707736e-05
8.76e-06
0.0012
0.5
2
96485.33289
1.131
1.115419203707736e-05
8.76e-06
0.0012
0.5
2
96485.33289
1.131
1.115419203707736e-05
8.76e-06
0.0012
0.5
2
96485.33289
1.131
1.115419203707736e-05
8.76e-06
0.0012
0.5
2
96485.33289
1.131
1.115419203707736e-05
8.76e-06
0.0012
0.5
[-4.82379368e-05]


In [0]:
for I_dis=I_dis_lst
    lam = petru_equation(n,F,A,Deff,c_int,l,I_dis);
    k = Deff./(lam).^2;
    eta_c = -Vt./n./beta.*(log(k./(k0.*a)));
    eta_c_sinh = -Vt./n./beta.*(asinh(k./(2.*k0.*a)));
    eta_a = -2.*Vt.*log(I_dis./ja0);
    eta_a_sinh = -2.*Vt.*asinh(I_dis./ja0./2);
    eta = eta_c + eta_a;
    eta_sinh = eta_c_sinh + eta_a_sinh;
    plot(eta,log(I_dis),'o','Color','Green','DisplayName','eta_tot_log');
    plot(eta_sinh,log(I_dis),'-p','Color','Blue','DisplayName','eta_tot_sinh');
    plot(eta_c,log(I_dis),'o','Color','Red','DisplayName','eta_c_log');
    plot(eta_c_sinh,log(I_dis),'^','Color','Blue','DisplayName','eta_c_sinh');
    plot(eta_a,log(I_dis),'o','Color','Black','DisplayName','eta_a_log');
    plot(eta_a_sinh,log(I_dis),'v','Color','Blue','DisplayName','eta_a_sinh');
    disp(['j_dis = ' num2str(I_dis) ' A/cm2'])
    disp(['Ceta = ' num2str(eta_c) ' V'])
    disp(['Ceta_sinh = ' num2str(eta_c_sinh) ' V'])
    disp(['lambda = ' num2str(lam) ' cm'])
    disp(['Aeta = ' num2str(eta_a) ' V'])
    disp(['Aeta_sinh = ' num2str(eta_a_sinh) ' V'])
    disp(['eta = ' num2str(eta) ' V'])
    disp(['eta_sinh = ' num2str(eta_sinh) ' V'])    
    disp(' ')
end

% hold on;plot(0:-20e-3:-0.5,(-38.922.*[0:-20e-3:-0.5]-17.291523),'--');hold off;
j=[500,1000,1500,2500,5000].*1e-6;
n=2.959-[2.71,2.683,2.663,2.621,2.515];
plot(-n,log(j),'o--');
plot(zeros(size(j)),log(j),'o--');hold off;